In [27]:
import pandas as pd
import numpy as np
from pingouin import cronbach_alpha
from scipy.stats import pearsonr, ttest_ind, f_oneway   

In [28]:
df = pd.read_csv("./resultados_avaliacao.csv")

In [29]:
def extrai_resposta_por_grupo(df, data_grupo1, data_grupo2, data_grupo3): 
    # Convertendo a coluna de datas para o formato datetime
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y %H:%M:%S')
    
    # Lista de heurísticas e mapeamento das respostas
    questoes = [
        'Visibilidade do estado do sistema', 
        'Correspondência entre o sistema e o mundo real', 
        'Controle e liberdade do usuário', 
        'Consistência e padrões', 
        'Prevenção de erros', 
        'Reconhecimento em vez de memorização', 
        'Flexibilidade e eficiência de uso', 
        'Design estético e minimalista', 
        'Ajude os usuários a reconhecer, diagnosticar e corrigir erros', 
        'Ajuda e documentação'
    ]
    
    mapeamento_respostas = {
        'Concordo Totalmente': 5,
        'Concordo': 4,
        'Indiferente': 3,
        'Discordo': 2,
        'Discordo Totalmente': 1
    }
    
    # Função auxiliar para extrair respostas por heurística
    def extrai_respostas(df):
        heuristicas_respostas = {heuristica: [] for heuristica in questoes}
        for heuristica in questoes:
            for coluna in df.columns:
                if heuristica in coluna:
                    contagem_respostas = df[coluna].value_counts()
                    respostas = []
                    for categoria, valor in mapeamento_respostas.items():
                        if categoria in contagem_respostas:
                            respostas += [valor] * contagem_respostas[categoria]
                    heuristicas_respostas[heuristica].append(respostas)
        return heuristicas_respostas

    # Agrupando os dados em três grupos com base nas datas
    grupo1 = df[df['data'].dt.strftime('%d/%m/%Y').isin(data_grupo1)]
    grupo2 = df[df['data'].dt.strftime('%d/%m/%Y').isin(data_grupo2)]
    grupo3 = df[df['data'].dt.strftime('%d/%m/%Y').isin(data_grupo3)]
    
    respostas_grupo1 = extrai_respostas(grupo1)
    respostas_grupo2 = extrai_respostas(grupo2)
    respostas_grupo3 = extrai_respostas(grupo3)
    
    # Análise de correlação entre os grupos para cada heurística
    for heuristica in questoes:
        respostas_g1 = respostas_grupo1[heuristica][0] if len(respostas_grupo1[heuristica]) > 0 else []
        respostas_g2 = respostas_grupo2[heuristica][0] if len(respostas_grupo2[heuristica]) > 0 else []
        respostas_g3 = respostas_grupo3[heuristica][0] if len(respostas_grupo3[heuristica]) > 0 else []
        
        # Convertendo para Series do pandas para análise
        serie_g1 = pd.Series(respostas_g1)
        serie_g2 = pd.Series(respostas_g2)
        serie_g3 = pd.Series(respostas_g3)
        
        print(f"\nAnálise para a heurística '{heuristica}':")
        
        # Correlação entre Grupo 1 e Grupo 2
        if len(serie_g1) > 1 and len(serie_g2) > 1:
            corr_12 = serie_g1.corr(serie_g2)
            print(f"Correlação entre Grupo 1 e Grupo 2: {corr_12:.2f}")

        # Correlação entre Grupo 2 e Grupo 3
        if len(serie_g2) > 1 and len(serie_g3) > 1:
            corr_23 = serie_g2.corr(serie_g3)
            print(f"Correlação entre Grupo 2 e Grupo 3: {corr_23:.2f}")

        # Correlação entre Grupo 1 e Grupo 3
        if len(serie_g1) > 1 and len(serie_g3) > 1:
            corr_13 = serie_g1.corr(serie_g3)
            print(f"Correlação entre Grupo 1 e Grupo 3: {corr_13:.2f}")

        # Teste ANOVA entre os três grupos se houver dados suficientes
        if len(serie_g1) > 1 and len(serie_g2) > 1 and len(serie_g3) > 1:
            f_stat, p_val_anova = f_oneway(serie_g1.dropna(), serie_g2.dropna(), serie_g3.dropna())
            sig = "significativa" if p_val_anova < 0.05 else "não significativa"
            print(f"ANOVA entre os três grupos: F={f_stat:.2f}, p-valor={p_val_anova:.4f} ({sig})")

In [30]:
data_grupo1 = ['23/10/2024'] # y
data_grupo2 = ['24/10/2024'] # x
data_grupo3 = ['22/10/2024', '25/10/2024'] # IHC

extrai_resposta_por_grupo(df, data_grupo1, data_grupo2, data_grupo3)


Análise para a heurística 'Visibilidade do estado do sistema':
Correlação entre Grupo 1 e Grupo 2: 0.91
Correlação entre Grupo 2 e Grupo 3: 0.71
Correlação entre Grupo 1 e Grupo 3: 0.64
ANOVA entre os três grupos: F=3.73, p-valor=0.0289 (significativa)

Análise para a heurística 'Correspondência entre o sistema e o mundo real':
Correlação entre Grupo 1 e Grupo 2: 0.79
Correlação entre Grupo 2 e Grupo 3: 0.85
Correlação entre Grupo 1 e Grupo 3: 0.64
ANOVA entre os três grupos: F=1.35, p-valor=0.2659 (não significativa)

Análise para a heurística 'Controle e liberdade do usuário':
Correlação entre Grupo 1 e Grupo 2: 0.69
Correlação entre Grupo 2 e Grupo 3: 0.89
Correlação entre Grupo 1 e Grupo 3: 0.74
ANOVA entre os três grupos: F=0.08, p-valor=0.9204 (não significativa)

Análise para a heurística 'Consistência e padrões':
Correlação entre Grupo 1 e Grupo 2: 0.69
Correlação entre Grupo 2 e Grupo 3: 0.83
Correlação entre Grupo 1 e Grupo 3: 0.64
ANOVA entre os três grupos: F=2.06, p-valor

/home/staubjoao/.local/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/staubjoao/.local/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
